In [3]:
! pip install haversine

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [4]:
import pandas as pd
from haversine import haversine

In [ ]:
# 역 좌표 데이터프레임 (stations_df)과 집 좌표 데이터프레임 (houses_df)이 있다고 가정합니다.
houses_df= pd.read_csv('/content/drive/MyDrive/23-1 빅데이터프로그래밍 기말과제/02. 1차가공데이터/2.3. 아파트단지 데이터셋 +단위가격 +주소.csv')
stations_df= pd.read_csv('/content/drive/MyDrive/23-1 빅데이터프로그래밍 기말과제/01. 로우데이터/지하철역 데이터셋.csv')

In [ ]:
houses_df.head()

,Unnamed: 0,단지명,단위면적가격,도로명,_GC_TYPE,_X,_Y
0,0,경남1,1747,언주로 110,정,127.053086,37.484492
1,1,개포자이,1782,개포로109길 69,정,127.076969,37.496079
2,2,개포래미안포레스트,3088,개포로 264,정,127.055667,37.477380
3,3,개포주공6단지,2289,개포로 516,정,127.069771,37.489347
4,4,래미안블레스티지,3808,선릉로 8,정,127.062645,37.482549


In [ ]:
stations_df.head()

,field1,field2,field3,field4,field5,field6,_GC_TYPE,_X,_Y
0,코레일,1호선,덕정,경기도 양주시 덕정동 350-14,경기도 양주시 화합로 1356,NaN,정,127.061805,37.844076
1,코레일,1호선,보산,경기도 동두천시 보산동 422,경기도 동두천시 평화로 2539,NaN,정,127.057237,37.914318
2,코레일,1호선,소요산,경기도 동두천시 상봉암동 126-3,경기도 동두천시 평화로 2925,NaN,정,127.061213,37.947953
3,코레일,1호선,동두천,경기도 동두천시 동두천동 245-210,경기도 동두천시 평화로 2687,NaN,정,127.054877,37.927825
4,코레일,1호선,지행,경기도 동두천시 지행동 424-1,경기도 동두천시 평화로 2285,NaN,정,127.055664,37.891876


In [ ]:
# 주의: 이 코드는 실행이 매우 오래 걸림 (약 5.5시간)

# 집과 역 간의 거리를 저장할 데이터프레임 생성
distances_df = pd.DataFrame(columns=['House', 'Station', 'Distance'])

# 모든 집에 대해 반복
for house_index, house_row in houses_df.iterrows():
    house_coords = (house_row['_Y'], house_row['_X'])  # 집 좌표
    
    # 모든 역에 대해 반복
    for station_index, station_row in stations_df.iterrows():
        station_coords = (station_row['_Y'], station_row['_X'])  # 역 좌표
        
        # 집과 역 간의 거리 계산
        distance = haversine(house_coords, station_coords)
        
        # 결과를 데이터프레임에 추가
        distances_df = distances_df.append({'House': house_row['단지명'],
                                            'Station': station_row['field3'],#역명
                                            'Distance': distance}, ignore_index=True)

In [ ]:
# 결과 출력
print(distances_df)

                  House    Station   Distance
0                   경남1         덕정  39.991349
1                   경남1         보산  47.795955
2                   경남1        소요산  51.539561
3                   경남1        동두천  49.296715
4                   경남1         지행  45.299695
...                 ...        ...        ...
1389295  한신아파트(101~102)         사평  11.875888
1389296  한신아파트(101~102)       삼성중앙   9.703237
1389297  한신아파트(101~102)        선정릉  10.158640
1389298  한신아파트(101~102)  흑석(중앙대입구)  14.305013
1389299  한신아파트(101~102)        봉은사   9.375040

[1389300 rows x 3 columns]


In [ ]:
distances_df.to_csv('/content/drive/MyDrive/2.4. allstn_to_allapt.csv', encoding='euc-kr', index=False)

In [ ]:
# df_distance에서 각 집당 가장 가까운 역과의 거리를 가지고 있는 row만 추출
df_min_distance = distances_df.loc[distances_df.groupby('House')['Distance'].idxmin()]

# 새로운 데이터프레임으로 저장
distances_df_min = df_min_distance.copy()

print(distances_df_min)

             House      Station  Distance
1178931   (1101-1)      구로디지털단지  0.194314
980794    (160-16)      고려대(종암)  0.663097
404502      (20-3)  어린이대공원(세종대)  0.573205
1314160     (60-1)           혜화  0.333281
312305   (685-103)       신대방삼거리  0.923399
...            ...          ...       ...
634708     힐스테이트청계           마장  0.671726
1082756  힐스테이트클래시안           신풍  0.506857
516202         힐캐슬           석수  0.593333
1201273    힐탑트레져1동           옥수  0.506213
1203863    힐튼빌리지2차          서울역  0.649476

[3300 rows x 3 columns]


In [ ]:
distances_df_min.to_csv('/content/drive/MyDrive/2.4. allstn_to_allapt_min.csv', encoding='euc-kr', index=False)